# Load config file (optional)

In [1]:
%%javascript

// Load MAPBOX token from local file
// this is not required, only for privacy purpose
require(['https://d3js.org/d3.v5.min.js'], d3 => {
    d3.json('config.json')
        .then(config => {
            window.config = {...config}
    })
})

<IPython.core.display.Javascript object>

# Deck.gl

In [3]:
%%javascript

// Define h3 and S2 namespaces.
// They are optional dependencies of deck.gl but the require will break if 
// they have not been defined
define('h3', () => ({}));
define('S2', () => ({}));

(element => {
    require(['h3', 'S2'], (h3, S2) => {
        require([
            'https://api.tiles.mapbox.com/mapbox-gl-js/v0.53.0/mapbox-gl.js',
            'https://cdn.jsdelivr.net/npm/deck.gl@7.2.3/dist.min.js',
            'https://d3js.org/d3.v5.min.js'
        ], (mapboxgl, deck, d3) => {
            console.log('config', config)
            const width = 800,
              height = 600
        
            const container = d3.select(element.get(0))
                .append('div')
                    .style('width', `100%`)
                    .style('height', `${height}px`)
                    .attr('id', 'viz-container')
                    
            const deckgl = new deck.DeckGL({
              container: container.node(),
              map: mapboxgl,
              mapboxApiAccessToken: window.config.MAPBOX_TOKEN, // <== ADD YOUR MAPBOX TOKEN HERE
              mapStyle: 'mapbox://styles/mapbox/dark-v9',
              latitude: 34.0323,
              longitude: -118.476,
              zoom: 9,
              minZoom: 5,
              maxZoom: 15,
              pitch: 40.5
            });

            let data = null;

            const COLOR_RANGE = [
              [1, 152, 189],
              [73, 227, 206],
              [216, 254, 181],
              [254, 237, 177],
              [254, 173, 84],
              [209, 55, 78]
            ];

            function renderLayer () {
              const options = {
                  'radius': 250,
                  'coverage': 1,
                  'upperPercentile': 100
              };

              const hexagonLayer = new deck.HexagonLayer({
                id: 'heatmap',
                colorRange: COLOR_RANGE,
                data,
                elevationRange: [0, 1000],
                elevationScale: 25,
                extruded: true,
                getPosition: d => d,
                opacity: 1,
                ...options
              });

              deckgl.setProps({
                layers: [hexagonLayer]
              });
            }

            Promise.all([ // we will load the current state of the bird fleet in the LA area
                d3.json('https://mds.bird.co/gbfs/los-angeles/free_bikes'),
                d3.json('https://mds.bird.co/gbfs/santamonica/free_bikes')
            ]).then(([response_la, response_sm]) => {
                console.log(response_la)
              data = [...response_la.data.bikes, ...response_sm.data.bikes]
                  .map(d => [Number(d.lon), Number(d.lat)]);
                console.log(data)
              renderLayer();
            });

        })         
    }) 
})(element);


<IPython.core.display.Javascript object>